In [ ]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

# <span style='color:red'>ch2. LLM 활용의 기본 개념</span>

# 1. LLM을 활용하여 답변 생성하기

## 1) ollama 이용한 로컬 LLM 이용

성능은 GPT, Claude 같은 모델보다 떨어지나, 개념 설명을 위해 open source 모델 사용

### ollama.com 다운로드 → 설치 → 모델 pull

- [deepseek다운로드] ollama pull deepseek-r1:1.5b (window키 + R → powershell)
- [deepseek실행] ollama run deepseek-r1:1.5b
- 다운위치 : /Users/teamkim/.ollama/


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="deepseek-r1:1.5b")

result = llm.invoke("What is the capital of the Korea?")
# result = llm.invoke('너 한글로 물어보면, 한글로 답변이 가능해?')
result  # 추론모델 <think>~<think>

### ollama.com 다운로드 → 설치 → 모델 pull

- [ollama다운로드] ollama pull llama3.2:1b
- [ollama실행] ollama run llama3.2:1b
- 다운위치 : /Users/teamkim/.ollama/
- llama : 공식적으로 한글지원이 안됨 (llama3.1 405b 한글지원 가능 → llama3.3 70b)
- exone : 공식적으로 한글지원


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of the Korea?")
result

In [ ]:
result.content

In [ ]:
result = llm.invoke("한국 수도는 어디예요?")
result

## 2) openai 활용

- pip install langchain-openai


In [7]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-mini")
# result = llm.invoke("What is the capital of the Korea?")
# result 에러이유 : OPENAI_API_KEY 환경변수 부재

In [ ]:
from dotenv import load_dotenv

# import os
load_dotenv()
# os.getenv("OPENAI_API_KEY")

In [9]:
# 코랩에서 OPENAI_API_KEY 읽어오기 (.env 못씀)
# 보안키 추가 후
# from google.colab import userdata
# userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    # openai_api_key=os.getenv("OPENAI_API_KEY")
)
llm.invoke("What is the capital of the Korea? Answer me in Korean")

In [11]:
# 모든 모델의 키가 OPENAI_API_KEY 는 아님
# Claude → Anthropic
# Azure → Azure OpenAI
# upstage, Bedrock : 에러 메시지 참조하여 환경변수 생성

In [12]:
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model="gpt-4.1-nano")
# llm.invoke("What is the capital of the Korea? Answer me in Korean")
# 에러를 내면, OPENAI_API_VERSION 환경변수가 필요하는 메세지

In [13]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
# llm.invoke("What is the capital of the Korea? Answer me in Korean")
# 에러 메시지를 봐도 환경변수 이름을 알 수 없음 → ChatAnthropic 검색 후, LangChain docs 에서 명시한 ANTHROPIC_API_KEY 이름의 환경변수 설정

# 2. 렝체인 스타일로 프롬프트 작성하기

- 프롬프트 : llm 호출시 쓰는 질문


In [14]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")
# llm.invoke(0) # error : PromptValue, str, or list of BaseMessages.

# ! 프롬프트 타입 : 스트링, PromptValue, BaseMessage리스트

## 1) 기본 프롬프트 템플릿 사용

- PromptTemplate을 사용하여 변수가 포함된 템플릿 작성


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate


llm = ChatOllama(model="llama3.2:1b")

prompt_template = PromptTemplate(
    # template="What is the {thing} of the {country}?", # {} 안의 값을 새로운 값으로 할당 가능
    # input_variables = ["country", "thing"]
    template="What is the capital of the {country}?",  # {} 안의 값을 새로운 값으로 할당 가능
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "Korea"})
print(prompt)
llm.invoke(prompt)

## 2) 메시지 기반 프롬프트 작성

- BaseMessage리스트
- BaseMessage 상속 받는 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

llm = ChatOllama(model="llama3.2:1b")

message_list = [
    # 예상 질물과 답변
    SystemMessage(content="You are a helpful assistant!"),
    
    HumanMessage(content="What is the capital of the Italy?"),
    AIMessage(content="The capital of Italy is Rome."),
    HumanMessage(content="What is the capital of the Korea?"),
    AIMessage(content="The capital of Korea is Seoul."),
    HumanMessage(content="What is the capital of the France?"),
]

llm.invoke(message_list)

In [ ]:
# BaseMessage list로 하면, 랭페인화도 않되고, ChatPromptTemplate 사용 불가

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

llm = ChatOllama(model="llama3.2:1b")

message_list = [
    # 예상 질물과 답변
    SystemMessage(content="You are a helpful assistant!"),
    
    HumanMessage(content="What is the capital of the Italy?"),
    AIMessage(content="The capital of Italy is Rome."),
    HumanMessage(content="What is the capital of the Korea?"),
    AIMessage(content="The capital of Korea is Seoul."),
    HumanMessage(content="What is the capital of the {country}?"),
]

from langchain_core.prompts import ChatPromptTemplate

chatPromptTemplate = ChatPromptTemplate.from_messages(message_list)

prompt = chatPromptTemplate.invoke({"country": "Korea"})

print(prompt)

## 3) ChatPromptTemplate 사용

- BaseMessage 리스트 → 튜플 리스트


In [ ]:
# 위의 BaseMessage 를 수정
chatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant!."),
        ("human", "What is the capital of the {country}?"),
    ]
)
country = input("어느나라 수도가 궁급하세요")
prompt = chatPromptTemplate.invoke({"country": country})
print("프롬프트 :", prompt)
result = llm.invoke(prompt)
result.content

In [ ]:
# 한국어로... - 정확성 떨어짐
chatPromptTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 수도를 맞추는 전문가야. 꼭 수도만 답변해라"),
        ("human", "{country}의 수도가 어디예요?"),
    ]
)
country = input("어느나라 수도가 궁급하세요")
prompt = chatPromptTemplate.invoke({"country": country})
print("프롬프트 :", prompt)

result = llm.invoke(prompt)
result

# 3. 답변 형식을 컨트롤하기

- invoke 실행결과 AIMessage() → String이나 JSON, 객체 : outputParser 이용 렝체인

## 1) 문자열 출력 파서 이용

- StrOutputParser를 사용하여, LLM출력(AIMessage)을 단순 문자열로 변환


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template="What is the capital of the {country}. Return the name of the city only.",
    input_variables=["country"],
)

# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country": "Korea"})
print("프롬프트 :", prompt)

result = llm.invoke(prompt)
print("llm 결과 :", result)

# 문자열 풀력 파서를 이용하여 llm 응답을 단순 문자열 변환
output_parser = StrOutputParser()
print("문자열 파서 결과 :", output_parser.invoke(result))

In [ ]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "Korea"})))

In [ ]:
# PromptTemplate(변수설정)  → ChatPromptTemplate(변수설정, system 과 모범답안 지정)
llm = ChatOllama(model="llama3.2:1b")

chatPrompt_template = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant with expertise in Seoul Korea"),
        ("human", "What is the capital of the {country}? Return the name of the city only."),
    ]
)

output_parser = StrOutputParser()

output_parser.invoke(llm.invoke(chatPrompt_template.invoke({"country": "Korea"})))

## 2) Json 출력 파서 이용


- json()으로 응답하기를 원하지만, 우선 어떤 형식으로 반환 되는지 확인
- {'name':'홍', 'age':22 ... } (json)) {'name':'홍', age:22} (dict)


In [ ]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""Give follwing information about {country}
        - Capital
        - Population
        - Language
        - Currency
        return it is JSON format and return the JSON dictionary only.
    """,
    input_variables=["country"],
)

prompt = country_detail_prompt.invoke({"country": "Korea"})
print(type(prompt), prompt)

# JSON output 파서
output_parser = JsonOutputParser()
ai_message = llm.invoke(prompt)

print(type(prompt), ai_message)

json_result = output_parser.invoke(ai_message)
print(type(json_result), json_result)

In [24]:
country_detail_prompt = PromptTemplate(
    template="""Give follwing information about {country}
        - Capital
        - Population
        - Language
        - Currency
        return it is JSON format and return the JSON dictionary only.
    """,
    input_variables=["country"],
)

output_parser = JsonOutputParser()
info = output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country": "Korea"})))

In [ ]:
type(info)

## 3) 구조화된 출력 사용

- Pydantic 모델을 사용하여, LLM출력을 구조화된 형식으로 받기 (JsonParser 보다 훨씬 안정적)
- Pydantic : 데이터 유효성 검사, 설정관리 

In [ ]:
from pydantic import BaseModel, Field

class User:
    def __init__(self, id, name, is_active=True):
        self.id = id
        self.name = name
        self.is_active = is_active

    # def __str__(self):
    #     return f"User(id={self.id}, name={self.name}, is_active={self.is_active})"

user = User(1, "홍길동")
print(user)

In [ ]:
from pydantic import BaseModel, Field

class User(BaseModel):
    # gt = 0:id>0, ge: 0:id >=0, lt : 0:id<0, le : 0:le<=0
    id: int = Field(gt=0, description="User ID")
    name: str = Field(min_length=2, description="User Name")
    is_active: bool = Field(default=True, description="User Active")
    
    def __str__(self):
        return f"User(id={self.id}, name={self.name}, is_active={self.is_active})"
    
user = User(id="1", name="홍길동")
user    

In [ ]:
country_detail_prompt = PromptTemplate(
    template="""Give follwing information about {country}
        - Capital
        - Population
        - Language
        - Currency
        return it is JSON format and return the JSON dictionary only.
    """,
    input_variables=["country"],
)

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

# 출력 형식 곤리
structedllm = llm.with_structured_output(CountryDetail)

output_parser = JsonOutputParser()

# output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country": "Korea"})))
# output_parser

info = structedllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
type(info)

In [ ]:
print(info)
print(info.capital)
print(info.population)
print(info.language)
print(info.currency)

In [ ]:
print('info.json :', info.model_dump_json()) # json()
print('info.dict :', info.model_dump()) # dict()

# 4. LCEL을 활용한 렝체인 생성하기

## 1) 문자열 출력 파서 사용
- invoke

In [ ]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",temparature=0) # 0: 일관된 답볍

# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of the {country}. Return the name of the city only.",
    input_variables = ["country"],
)

output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

## 2) LCEL을 사용한 간단한 체인 구성

- 파이프연산자(|) 사용


In [ ]:
# 프롬프트 템플릿 → llm → 출력파서를 연결
capital_chain = prompt_template | llm | output_parser

# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

In [ ]:
type(capital_chain)

## 3) 복합 체힌 구성

- 여러단계의 추론이 필요한 경우 (체인 연결)

In [ ]:
# 나라 설명 → 나라명
from langchain_core.prompts import ChatPromptTemplate

country_propmt = PromptTemplate(
    template = """Guess the name of the country based on the following information: : {information} 
    Return the name of the country only""",
    input_variables=["information"]
)

output_parser.invoke(llm.invoke(country_propmt.invoke({"information": "The country is very famous for it's wine"})))


In [ ]:
# 나라명 추측 체인 생성
country_chain = country_propmt | llm | output_parser
print(type(country_chain))

country_chain.invoke({"information": "The country is very famous for it's wine"})

In [ ]:
# 나라설명 → (나라명 →) 그 나라 수도
final_chain = country_chain | capital_chain
# type(final_chain)
final_chain.invoke({"information": "The country is very famous for it's wine"})

In [ ]:
# 나라설명 → 나라명 → 수도 체인 생성 
final_chain = {"country":country_chain} | capital_chain

final_chain.invoke({"The country is very famous for it's wine"})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

final_chain = {"information": RunnablePassthrough()} | {"country":country_chain} | capital_chain

final_chain.invoke({"information": "The country is very famous for it's wine"})

In [ ]:
# 프롬프트 템플릿에 변수가 2개인 경우
# 나라 설명 → 나라명
from langchain_core.prompts import ChatPromptTemplate

country_propmt = PromptTemplate(
    template = """Guess the name of the country in the {continent} based on the following information: : {information} 
    Return the name of the country only""",
    input_variables=["information", "continent"]
)

# output_parser.invoke(llm.invoke(country_propmt.invoke({"information": "The country is very famous for it's wine","continent":"Europe"})))
country_chain = country_propmt | llm | output_parser

country_chain.invoke({"information": "The country is very famous for it's wine","continent":"Europe"})

In [ ]:
final_chain = {"country":country_chain} | capital_chain

final_chain.invoke({"information": "The country is very famous for it's wine","continent":"Europe"})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

final_chain = {"information": RunnablePassthrough()} | {"country":country_chain} | capital_chain

final_chain.invoke({"information": "The country is very famous for it's wine"})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

final_chain = {"information": RunnablePassthrough()} | {"country":country_chain} | capital_chain

final_chain.invoke({"information": "The country is very famous for it's wine"})

In [1]:
# Q. 나라명 → (제일 유명한 음식 →) 음식의 레시피
# 다음의 생성형 ai를 렝체인으로 구현하시오.
# ▪ 원하는 모델을 이용하여 나라를 입력하면 해당 나라의 가장 유명한 음식을 추천하는 렝체인을 구현
# (food_chain))
# ▪ 음식을 입력하면 레시피를 생성하는 렝체인을 구현(recipe_chain)
# ▪ 위의 두 체인을 연결하여, 나라이름을 입력받아, 해당 나라의 가장 유명한 음식의 레시피를 생성하는 렝체인 프로그램을 구현하시오.

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama3.2:1b",temparature=0) # 0: 일관된 답변


country_input = input("Enter the country name: ")

food_prompt = PromptTemplate(
    template = """What is the most famous food in {country_name}?
    Return only the name of the food.""",
    input_variables=["country_name"]
)


food_chain = food_prompt | llm | StrOutputParser()

food_result =food_chain.invoke({"country_name": country_input})
print(food_result)


recipe_prompt = PromptTemplate(
    template = """What is the recipe for {food_name}?
    Return only the recipe.""",
    input_variables=["food_name"]
)

recipe_chain = recipe_prompt | llm | StrOutputParser()

recipe_result = recipe_chain.invoke({"food_name": food_result})
print(recipe_result)

Bibimbap.
Ingredients:

* 1 cup of mixed vegetables (such as bean sprouts, zucchini, carrots, and spinach)
* 2 cups of cooked white rice
* 1/4 cup of toasted sesame seeds
* 1/4 cup of chopped green onions
* 2 tablespoons of gochujang sauce
* 2 tablespoons of soy sauce
* 1 tablespoon of rice vinegar
* 1 teaspoon of ground black pepper
* 1/2 teaspoon of crushed red pepper (optional)
* 2 eggs, beaten
* 1 cup of diced zucchini
* 1 cup of diced cucumber
* 1 cup of diced carrots
* 1/4 cup of chopped scallions
* 1 tablespoon of toasted sesame oil

Instructions:

1. Cook the rice and let it cool.
2. In a large bowl, combine the mixed vegetables, cooked white rice, toasted sesame seeds, green onions, gochujang sauce, soy sauce, rice vinegar, black pepper, crushed red pepper (if using), and beaten eggs.
3. Add the diced zucchini, cucumber, and carrots to the bowl and toss gently.
4. Ladle the Bibimbap onto a plate and top with additional sesame seeds, green onions, and scallions if desired.
5. S